# Archaic to Modern Italian with Context Learning

In [ ]:
# Import Datases to work with Transformers by Hugging-Face
import torch
from tqdm.auto import tqdm
# Imports for Transformers
from datasets import Dataset
from transformers import AutoTokenizer  # Datasets
import pandas as pd
from datasets.features import Value, Features

## Globals

# Promposed Models
* google/flan-t5-small - google/mt5-small (text2text model) ::NO_WORK
* google/gemma-3-1b-it (LLM) 🚀
* sapienzanlp/Minerva-1B-base-v1.0 🇮🇹 (LMM)
* Helsinki-NLP/opus-mt-itc-itc (Machine Translation) 🏆 - use OpusPrompt 
* FacebookAI/xlm-roberta-base (fill-mask)

In [ ]:
DATASET = "dataset.csv"
FEATURES = Features(
    {
        "Author": Value(dtype="string"),
        "Date": Value(dtype="string"),
        "Region": Value(dtype="string"),
        "Sentence": Value(dtype="string")
    }
)
NET = "google/gemma-3-1b-it"
BS = 8
DEVICE = ('cuda' if torch.cuda.is_available() else "cpu")

## Define Network Pipline


In [ ]:
tokenizer = None
def ft5_std_map(examples):
    return tokenizer(
        [f"Traduci dal volgare all’italiano moderno: {example}" for example in examples["Sentence"]],  
        padding=True, 
        max_length=128)

def gemma_1b_map(examples):
    chat = tokenizer.apply_chat_template([
    [
        {"role": "system",   "content": "Sei un traduttore esperto di Italiano Antico "},
        {"role": "user",     "content": "Traduci 'La corte era in gran fermento.' in Italiano Moderno"},
        {"role": "assistant","content": "Italiano Antico: 'La corte era in gran fermento.' Italiano Moderno: 'La corte era molto agitata.'"},
        {"role": "user",      "content": f"Traduci '{example}' in Italiano Moderno"},
        {"role": "assistant", "content": ""}
    ] for example in examples["Sentence"] ], 
        tokenize=True,
        add_generation_prompt=True,
        return_dict=True,
        padding=True,         # <== aggiunge zeri per rendere le sequenze uguali
        truncation=True,      # <== taglia sequenze troppo lunghe
        max_length=250,       # (opzionale) puoi specificare una lunghezza massima
        return_tensors="pt"
    )

    return chat

def mask_std_map(examples):
    return tokenizer(
        [f"Old Italian: {example} Modern Italian [MASK]" for example in examples["Sentence"]],  
        padding=True, 
        max_length=128)

    


In [ ]:
match NET:
    
    case "google/flan-t5-small" | "google-t5/t5-small" |  "google/mt5-small":
        from transformers import T5Tokenizer, T5ForConditionalGeneration
        tokenizer = T5Tokenizer.from_pretrained(NET)
        model = T5ForConditionalGeneration.from_pretrained(NET, device_map=DEVICE, torch_dtype=torch.float16)
        tr = ft5_std_map

        params = {
            
            "max_new_tokens": 120,
            "do_sample":True,
            "top_k":50,            # aumento della diversità controllando le parole candidate
            "top_p":0.90,          # campionamento nucleus per ulteriori controlli sulla varietà
            "temperature":1.0,     # riduce la casualità e aumenta la coerenza
            "repetition_penalty":1.0,  # penalizza ripetizioni
            "num_return_sequences":10,  # numero di risposte generate
            "pad_token_id":tokenizer.eos_token_id  # evita warning se manca un token di padding
        }
    case "google/mt5-base":
        from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
        tokenizer = AutoTokenizer.from_pretrained(NET)
        model = AutoModelForSeq2SeqLM.from_pretrained(NET, device_map=DEVICE, torch_dtype=torch.float16)
        tr = ft5_std_map

        params = {
            
            "max_new_tokens": 120,
            "do_sample":True,
            "top_k":10,            # aumento della diversità controllando le parole candidate
            "top_p":0.90,          # campionamento nucleus per ulteriori controlli sulla varietà
            "temperature":1.0,     # riduce la casualità e aumenta la coerenza
            "repetition_penalty":1.0,  # penalizza ripetizioni
            "num_return_sequences":10,  # numero di risposte generate
            "pad_token_id":tokenizer.eos_token_id  # evita warning se manca un token di padding
        }
    case "google/gemma-3-1b-it":
        from transformers import BitsAndBytesConfig, Gemma3ForCausalLM, AutoTokenizer
        tokenizer = AutoTokenizer.from_pretrained(NET)
        quantization_config = BitsAndBytesConfig(load_in_8bit=True)
        model = Gemma3ForCausalLM.from_pretrained(NET, device_map=DEVICE, quantization_config=quantization_config)
        tr = gemma_1b_map
        params = {
            
            "max_new_tokens": 120,
            "do_sample":True,
            "top_k":10,            # aumento della diversità controllando le parole candidate
            "top_p":0.90,          # campionamento nucleus per ulteriori controlli sulla varietà
            "temperature":1.0,     # riduce la casualità e aumenta la coerenza
            "repetition_penalty":1.0,  # penalizza ripetizioni
            "num_return_sequences":10,  # numero di risposte generate
            "pad_token_id":tokenizer.eos_token_id  # evita warning se manca un token di padding
        }
    
    case "FacebookAI/xlm-roberta-base":
        
        from transformers import AutoTokenizer, AutoModelForMaskedLM
        tokenizer = AutoTokenizer.from_pretrained(NET)
        model = AutoModelForMaskedLM.from_pretrained(NET)
        tr = mask_std_map
        params = {
            
            "max_new_tokens": 120,
            "do_sample":True,
            "top_k":10,            # aumento della diversità controllando le parole candidate
            "top_p":0.90,          # campionamento nucleus per ulteriori controlli sulla varietà
            "temperature":1.0,     # riduce la casualità e aumenta la coerenza
            "repetition_penalty":1.0,  # penalizza ripetizioni
            "num_return_sequences":10,  # numero di risposte generate
            "pad_token_id":tokenizer.eos_token_id  # evita warning se manca un token di padding
        }

    case _:
        raise Exception(f"Rete {NET} non testabile")
        

# Dataset

In [76]:
hf = Dataset.from_csv(DATASET, features=FEATURES).shuffle(seed=42)

In [77]:
tokenized =hf.map(tr, batched=True)

Map:   0%|          | 0/97 [00:00<?, ? examples/s]

In [78]:
print(tokenized.column_names)
for idx, s in enumerate(tokenized.take(5), 1) or "google-5/t5-small":
    print(f"sample n°{idx}: {s}")
    print(f"Decode ids: {tokenizer.decode(s["input_ids"], attention_mask=s["attention_mask"], skip_special_tokens=True)}")

['Author', 'Date', 'Region', 'Sentence', 'input_ids', 'attention_mask']
sample n°1: {'Author': 'Guido da Pisa', 'Date': '1337', 'Region': 'tosc.', 'Sentence': "Ed ecco di subito tutta questa turba degli uccelli si levò a volo dietro all'aquila", 'input_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 105, 2364, 107, 1869, 236747, 723, 3031, 196848, 15112, 1071, 1001, 168248, 5307, 2486, 236743, 108, 2035, 25593, 1287, 756, 4967, 50508, 6933, 528, 9085, 28196, 236748, 7085, 528, 168248, 13806, 236748, 106, 107, 105, 4368, 107, 64835, 236748, 5307, 2486, 236787, 756, 4967, 50508, 6933, 528, 9085, 28196, 236748, 7085, 168248, 13806, 236748, 236787, 756, 4967, 50508, 6933, 25965, 93160, 805, 7085, 106, 107, 105, 2364, 107, 2035, 25593, 1287, 756, 4675, 199304, 1001, 79727, 67169, 24903, 7909, 3604, 21751, 98474, 11981, 2083, 16866, 237493, 496, 228821, 157595, 784, 236789, 42709, 5657, 236789, 528, 168248, 13806, 236748, 106

In [79]:
output = []
model= model.eval()
with torch.no_grad():
    tokenized.set_format(type='torch', columns=['input_ids', 'attention_mask'])
    loader = torch.utils.data.DataLoader(tokenized, batch_size=BS)
    size = len(tokenized)
    for batch in tqdm(loader, dynamic_ncols=True, leave=True):
        
        input_ids=batch["input_ids"].to(DEVICE)
        attention_mask=batch["attention_mask"].to(DEVICE)

        pred = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        **params  
        )
        output.extend(pred)

In [80]:
df = pd.DataFrame(columns=["Original", "Translation(Generated)", "Evaluation"]) 
for idx, (y, y_pred) in enumerate(zip(hf, pred), 1):
    print(f"Sample n°{idx}")
    print(f"User say:\n Prompt + {y["Sentence"]}")
    response = tokenizer.decode(y_pred , skip_special_tokens=True)
    print(f"Model say:\n {response}")
    print(f"=======End Sentence n°{idx}=======")

    df.loc[len(df), "Original"] = y["Sentence"]
    df.loc[len(df) -1, "Translation(Generated)"] = response

df.to_csv(f"./Translation model({NET.split('/')[0]}).tsv", index=False, quotechar="\'", encoding='utf-8', sep="\t")


Sample n°1
User say:
 Prompt + Ed ecco di subito tutta questa turba degli uccelli si levò a volo dietro all'aquila
Model say:
 user
Sei un traduttore esperto di Italiano Antico 

Traduci 'La corte era in gran fermento.' in Italiano Moderno
model
Italiano Antico: 'La corte era in gran fermento.' Italiano Moderno: 'La corte era molto agitata.'
user
Traduci 'Acciocché quegli, il quale ora per le sue gran reità è feroce e onorevole, egli d'ogni male afflitto e tormentato della impietà verso il mio padre.' in Italiano Moderno
model

model
Acciocché quell'uomo, il quale ora per le sue grandi ricchezze è feroce e onorevole, egli combatte ogni male e ogni tormento che la sua pietà verso il mio padre lo affligge.

Oppure, un'altra traduzione, leggermente più poetica:

Acciocché quell'uomo, che ora per le sue ricchezze è feroce e onorevole, si ribella a ogni male e a ogni tormento che la sua pietà verso il mio padre lo affligge.

Scegli l'op
=======End Sentence n°1=======
Sample n°2
User say:
 P